In [279]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import csv
import numpy as np

In [280]:
RESULT_PATH = 'D:/DA/문화공모전/Crawling_data/'
now = datetime.now()  # 파일이름 현 시간으로 저장하기


maxpage = input("최대 출력할 페이지수 입력하시오: ex)blog_max:100, news:400")
query = input("검색어 입력: ")
s_date = input("시작날짜 입력(2019.01.01):")  # 2019.01.01
e_date = input("끝날짜 입력(2019.04.28):")  # 2019.04.28


In [281]:
def get_blogs(n_url):
    title_MainText = []
    
    #스마트 에디터 사용 여부에 따라 블로그 주소, HTML 바뀜.
    if n_url.startswith("https://blog.naver.com"):
        
        #블로그 주소 형식 변환
        n_url = n_url.replace('?Redirect=Log&logNo=', '/')
        n_url = n_url[0:8] + 'm.' + n_url[8:]
        
    else:
        #블로그 주소 형식 변환
        n_url = re.sub("^https?://(.+)[.]blog[.]me", r'https://m.blog.naver.com/\1', n_url)   
    #파싱
    print(n_url)
    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')
    #제목
    title = bsoup.select_one('div.se-component')
    if title == None:
        title_MainText.append(np.nan)
    else:
        title = title.find_all('p')[0].get_text()
        title_MainText.append(title)
    #title = bsoup.select_one('div.se-module').find_all('p')[0].get_text()
    #title_MainText.append(title)
    #본문

    main_text = bsoup.select_one('div.se-main-container')
    
    if main_text == None:
        title_MainText.append(np.nan)
    else :
        main_text = bsoup.select_one('div.se-main-container').find_all('p')
        results = ''
        for item in main_text:
            results += item.get_text()
        title_MainText.append(results)
    
    return title_MainText
    
    """
    main_text = bsoup.select_one('div.se-main-container').find_all('p')
    results = ''
    for item in main_text:
        results += item.get_text()
    title_MainText.append(results)
    return title_MainText
    
    """


In [282]:
def crawler(maxpage, query, s_date, e_date):
    s_from = s_date.replace(".", "")
    e_to = e_date.replace(".", "")
    page = 1
    maxpage_t = (int(maxpage) - 1) * 10 + 1  # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    f = open(RESULT_PATH + "%s.txt" % str(query), 'w', encoding='utf-8')

    while page < maxpage_t:

        print(page)

        url = "https://search.naver.com/search.naver?where=post&query=" + query + "&st=sim&date_from=" + s_date + "&data_option=8&data_to=" + e_date + "&nso=p%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page) + "where=post&srchby=all&sm=tab=pge&post_blogurl_without=tistory.com"    
        
        req = requests.get(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')

        for urls in soup.select("._sp_each_url"):
            if 'blog.me' in urls["href"] or 'blog.naver.com' in urls["href"]:
                news_detail = get_blogs(urls["href"])
                f.write("{}\t{}\n".format(news_detail[0], news_detail[1]))  # new style
                
            else:
                continue
        page += 10
        
    f.close()
            



In [283]:
crawling_data = crawler(maxpage, query, s_date, e_date)  # 검색된 네이버뉴스의 기사내용을 크롤링합니다.

data = pd.read_csv(RESULT_PATH + '%s.txt' % str(query), sep='\t', header=None, error_bad_lines=False, engine='python', encoding='utf-8')
df = pd.DataFrame(data)
df.columns = ['title','contents']
df.to_csv(RESULT_PATH + "%s.csv" % str(query), encoding='utf-8', index=False)



1
https://m.blog.naver.com/nailstan/221984265221
https://m.blog.naver.com/anndam/221958210000
https://m.blog.naver.com/12gojeju/221981459242
https://m.blog.naver.com/rladbrud8791/221973938583
https://m.blog.naver.com/suk4408/221977277753
https://m.blog.naver.com/okkmskim/221962436833
https://m.blog.naver.com/kji206/221970736331
https://m.blog.naver.com/hnmh2332/221857827022
https://m.blog.naver.com/hp0210/221783398461
https://m.blog.naver.com/goystour/221971831988
11
https://m.blog.naver.com/bb00dd/221864221162
https://m.blog.naver.com/spa7981/221966621153
https://m.blog.naver.com/anndam/221906699769
https://m.blog.naver.com/nailstan/221953867982
https://m.blog.naver.com/juco9075/221982460269
https://m.blog.naver.com/suk4408/221933076678
https://m.blog.naver.com/jbm993/221790956137
https://m.blog.naver.com/okkmskim/221899061524
https://m.blog.naver.com/kji206/221973092963
21
https://m.blog.naver.com/hp0210/221762087606
https://m.blog.naver.com/goystour/221970303730
https://m.blog.naver

181
https://m.blog.naver.com/sweetservant/221924840109
https://m.blog.naver.com/cafeinfofam/221912867051
https://m.blog.naver.com/kangminsu98/221974807176
https://m.blog.naver.com/handbird2/221915467063
https://m.blog.naver.com/rhkwkdrhkdrh/221981210191
https://m.blog.naver.com/pressinfo/221958648907
https://m.blog.naver.com/ejp8702/221969279858
https://m.blog.naver.com/krh0708/221924915748
https://m.blog.naver.com/wlsrud1318/221932944138
https://m.blog.naver.com/rhkrwlsdud45/221919143730


Skipping line 82: Expected 2 fields in line 82, saw 7


In [284]:
#csv 읽을 때 parameter
#data= pd.read_csv('PATH',engine='python', encoding='utf-8')
#data.dropna()